# Python Class Final Project
## Main Functions:
- Spell Check: A.Based on the inputted paragraph, do the spell check, and auto correct the word based on the 20,000 English words; B.Disply the corrected paragraph; C.Display the mis-spelling words and their replacement;


- Dictionary Lookup: A.Based on the 1'st inputted word, look up the dictionary, and display its meaning based on the definition of 45,000 English words in json file; B.If the input word doesn't exist in the dictionary, find the closed match.

## File Structure:
- root directory:
 1. app_frontend.py - The frontend program mainly using Flask;
 2. app_spell_check.py - Based on 20,000 English words flat file, using edit distance algorithm to correct the spelling;
 3. app_dictionary.py - Based on the definition of 45,000 English words in json file, disply the definition for a giving word; if it doesn't exist, using difflib.get_close_matches library to find the closed word.
- templates directory:
 1. index.html: html file with 3 parameters: {{text_input}}, {{text_output}} and {{error_output}}
- static directory:
 1. main.css: CSS file for index.html;
 2. 20K.txt: 20,000 English words flat file;
 3. data.json: 45,000 English words with their definitions in json format.

In [ ]:
# index.html

<html lang="en">
  <title> Qi Jing's Python Project</title>
 <head>
    <link href="../static/main.css" rel="stylesheet">
  </head>
    <body>
      <div class="container">
        <h1>Spell Check for Python Project</h1>
        <h3>Please Type a Paragraph</h3>
        <form method="POST" action="/">
          <textarea title="You Must Enter a Sentence !" placeholder="Enter a Sentence" name="text_area" rows="6"
            cols="60">{{text_input}}</textarea> <br><br>
          <input type="submit" name="action" value="Check Spelling">
          <input type="submit" name="action" value="Dictionary">
        </form>
        <textarea title="This is the Result !" name="result_area" rows="12" cols="80" 
         style="background:#99FFFF; color:purple;">{{text_output}}</textarea> 
        <textarea title="Result !" name="result_area" rows="12" cols="30" 
         style="background:yellow; color:red;">{{error_output}}</textarea> <br><br>
        <a href="https://github.com/jingqi8/CIS-024C/blob/master/Project.ipynb" 
         target="_blank">View Source Code and Documentation</a>
        <br><br>
        <a href="mailto:jingqi@yahoo.com?Subject=Python%20Class" target="_top">Send Feedback to Qi Jing at jingqi@yahoo.com</a><br><br>
        <a href="https://www.zillow.com/profile/Qi-Jing/" target="_blank">View My Professional Website</a>
      </div>
    </body>
</html>


In [ ]:
# app_fronend.py

from flask import Flask, render_template, request
from app_spell_check import *
from app_dictionary import *

app=Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def index():
    text_inputs=""
    text_outputs=""
    error_outputs=""
    if request.method == 'POST':
       	text_inputs=request.form["text_area"]
    if request.form["action"] == "Check Spelling":
        results = spell_check(text_inputs)
        text_outputs = results[0]
        error_outputs = results[1]
    else:
        first_word = text_inputs.split()[0]
        text_outputs = dictionary_lookup(first_word)

    return render_template("index.html", text_input=text_inputs, text_output=text_outputs, error_output=error_outputs)

if __name__ == '__main__':
    app.run(host='0.0.0.0',port=80, debug=True)



In [ ]:
# app_spell_check.py

f = open("./static/20K.txt", "r")        # Open the file contains 20,000 English words
words = f.read()                # Read it into words varaible
f.close()
words_list = words.split()      # Convert into a List
words_dictionary = {}           # A Dictionary to store already corrected word for performance reason

def spell_check(sentence):
   result = ""
   correct_result = ""
   sentence_list = sentence.split()
   words_dictionary.clear()
   for word in sentence_list:
       word_lower = word.lower()
       if word[-1].isalpha() != True:   # Need to remove the non-alpha character    
          word_lower = word_lower[0:-1]

       word_back = auto_correction(word_lower)

       if word.istitle() == True:       # Need to change back the first upper case
          word_back = word_back.title()
       if word[-1].isalpha() != True:   # Need to add the non-alpha character at end
          word_back = word_back + word[-1]
       result = result + word_back + " "

       if len(words_dictionary) == 0:
          correct_result = "EXCELLENT JOB!\n\nTHERE IS NO ANY ERROR."
       else:
          correct_result = "HERE ARE THE CORRECTIONS:\n\n"
          for item in words_dictionary:
             correct_result = correct_result + item + " ---> " + words_dictionary[item] + "\n\n"

   return [result, correct_result]

# Correct the word based on the List of 20K words and the min edit distance
def auto_correction(word):
    if word in words_list or word.isdigit() == True:  # Check if this word is already in the Words List
        result = word
    elif words_dictionary.get(word) != None:          # Check if this word is already corrected before
        result = words_dictionary[word]
    else:
        distance = 9999             # Initial the distance
        result = ""                 # Initial the result
        for w in words_list:        # Calculate the min edit distance based on the 20K word List
            dis = edit_distance(word, w)
            if distance > dis:
                distance = dis
                result = w      
        words_dictionary[word] = result       # Store the correction for next time use

    return result

# Function to calculate the distance between 2 words
def edit_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

# ---------------------- Main Program -----------------------------------------------------
if __name__ == "__main__":
# === Sentence correction ==========
   sentence = raw_input("Please enter a Sentence: ")
   print spell_check(sentence)


In [ ]:
# app_dictionary.py

import json
from difflib import get_close_matches

data = json.load(open("./static/data.json", "r"))
def dictionary_lookup(word):
    msg = ""
    close_w = ""
    out = ""
    w = word.lower()
    if w in data:
        msg = data[w]
    else:
        if len(get_close_matches(w, data.keys())) > 0:
           close_w = get_close_matches(w, data.keys())[0]
           msg = data[close_w]

    if msg == "":              # Nothing found or matched
        return "THE 1'ST WORD DOESN'T EXIST OR MATCH ANY CLOSED WORD. PLEASE DOUBLE CHECK!"
    else:
        if close_w != "":      # Found the closed match
           out = "WE FOUND THE MOST CLOSED WORD: " + close_w + "\n\n"
        else:                  # Found in the dictionary
           out = "THE MEANING OF THE 1'ST WORD: " + word + "\n\n" 
        if type(msg) == list:
           for item in msg:
               out = out + "-" + item + "\n\n"
        else:
           out = out + msg
        return out

# ---------------------- Main Program -----------------------------------------------------
if __name__ == "__main__":
   word = ""
   while word != "exit":
      word = raw_input("Enter a Word ('exit' to End): ")
      print dictionary_lookup(word)
        

In [ ]:
# main.css

html, body {
  height: 100%;
  margin: 0;
}

.container {
  margin: 0 auto;
  width: 100%;
  height: 100%;
  background-color: #006666;
  color: #e6ffff;
  overflow:hidden;
  text-align: center;
}

.container h1 {
  font-family: Arial, sans-serif;
  font-size: 30px;
  color: #e6ffff;
  margin-top: 80px;
}

.container h3 {
  color: red;
}

.container form {
  margin: 20px;
}

.container input {
  display: inline;
  width: 170px;
  height: 30px;
  background-color: steelblue;
  color: yellow;
  margin: 3px;
}


In [ ]:
# 20K.txt (sample)

......
page
my
has
search
free
but
our
one
other
do
no
information
......

In [ ]:
# data.json (sample)

{....
"acidity": 
 ["The state of being acid that is of being capable of transferring a hydrogen ion in solution."], 
"acidity degree": 
 ["The amount of acid present in a solution, often expressed in terms of pH."], 
"acid": 
 ["A compound capable of transferring a hydrogen ion in solution.", 
  "Being harsh or corrosive in tone.", "Having an acid, sharp or tangy taste.", 
  "A powerful hallucinogenic drug manufactured from lysergic acid.", 
  "Having a pH less than 7, or being sour, or having the strength to neutralize alkalis."], 
"acid rain": 
 ["Rain having a pH less than 5.6."], 
 ....
}